# Предсказание результатов матча в игре Dota 2 по событиям первых 5 минут игры

## Подготовка данных
Загрузим таблицу с признаками из файла features.csv и удалим признаки, связанные с итогами матча.
Поместим в X набор признаков, в y - столбец с целевой переменной radiant_win(победа команды radiant)

In [39]:
import pandas
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
import time
import datetime

data = pandas.read_csv('features.csv', index_col='match_id')
X = data.loc[:, 'start_time':'dire_first_ward_time']
y = data['radiant_win']

Проверим, есть ли у нас признаки с пропусками значений, посчитаем количество пропусков

In [2]:
nan_count = X.isnull().sum()
print(nan_count[nan_count != 0])

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64


Пропуски в датасете связаны с особенностью выборки, ведь мы анализируем только первые 5 минут матча. В первые 5 минут могут не произойти некоторые события(первое убийство героя - first blood, покупка предмета bottle командой radiant - radiant_bottle_time, покупка предмета ward командой dire - dire_first_ward_time)

Заменим все пропущенные значения в этом датасете на нули:

In [3]:
X.fillna(inplace=True, value=0)

Разделим обучающую выборку на 5 блоков с помощью генератора KFold

In [4]:
kf = KFold(n_splits=5, random_state=1, shuffle=True)

for train, test in kf.split(X):
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]

Теперь попробуем обучить два разных классификатора и сравним их качество предсказаний

## Подход 1 - градиентный бустинг

Обучим наш классификатор для различного количества деревьев. Будем считать качество классификатора по метрике ROC_AUC и время, в течение которого настраивалась модель

In [37]:
last_score = 0.5  # качество случайного распределения
for trees in range(10, 110, 10):
    gbm = GradientBoostingClassifier(n_estimators=trees, subsample=1.0, max_depth=3,
                                     random_state=241)
    gbm_score = 0
    start_time = datetime.datetime.now()
    gbm.fit(X_train, y_train)
    pred = gbm.predict_proba(X_test)[:, 1]
    gbm_score = roc_auc_score(y_test, pred)
    time = (datetime.datetime.now() - start_time).total_seconds()
    print(f'{trees:3d} деревьев | качество: {gbm_score:5.3f} | время обучения: ',
          f'{time:5.1f} c | прирост качества в сек: {((gbm_score-last_score)/time):7.5f}')
    last_score = gbm_score

 10 деревьев | качество: 0.667 | время обучения:   19.8 c | прирост качества в сек: 0.00844
 20 деревьев | качество: 0.684 | время обучения:   38.7 c | прирост качества в сек: 0.00043
 30 деревьев | качество: 0.691 | время обучения:   58.8 c | прирост качества в сек: 0.00012
 40 деревьев | качество: 0.695 | время обучения:   98.8 c | прирост качества в сек: 0.00004
 50 деревьев | качество: 0.698 | время обучения:   95.1 c | прирост качества в сек: 0.00003
 60 деревьев | качество: 0.701 | время обучения:  115.9 c | прирост качества в сек: 0.00003
 70 деревьев | качество: 0.703 | время обучения:  129.4 c | прирост качества в сек: 0.00002
 80 деревьев | качество: 0.705 | время обучения:  171.0 c | прирост качества в сек: 0.00001
 90 деревьев | качество: 0.706 | время обучения:  174.7 c | прирост качества в сек: 0.00001
100 деревьев | качество: 0.707 | время обучения:  215.5 c | прирост качества в сек: 0.00000


При увеличении количества деревьев увеличивается качество классификации и время обучения. Видим, что брать количество деревьев больше 30 неоптимально по затратам времени. 
Чтобы ускоить обучение, можно либо уменьшать размеры подвыборки(параметр subsample), либо уменьшить максимальную глубину дерева(параметр max_depth)

## Подход 2 - логистическая регрессия

Для логистической регрессии важно выполнить предварительное масштабирование признаков с помощью StandartScaler

In [45]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

Обучим модель 

In [ ]:
logreg = LogisticRegression(penalty='l2', )

1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?
2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?
3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).
4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.
5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?
6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).